In [ ]:
import os
import cv2
import random
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
from datetime import datetime
import matplotlib.pyplot as plt
from skimage.transform import resize
get_ipython().run_line_magic('matplotlib', 'inline') ###
################################################################################
import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
################################################################################
from keras import layers,utils
from keras.datasets import cifar100
from keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Activation, Flatten,Conv2D,MaxPooling2D, MaxPool2D,Add, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D,LeakyReLU, Dense, InputLayer,Dropout, Activation, Flatten,Input, BatchNormalization, Conv2D, MaxPool2D, GlobalMaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, MobileNet, ResNet50, InceptionV3, Xception, VGG19,ResNet101
# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
################################################################################
import gc

In [ ]:
# Global_Variables

# Path for the dataset directory which contains directory for each class (GA , Intermediate , Normal , Wet)
dataset_path = '../AMD_DataSet'

# Path to store the splitted dataset in it as (train , val , test)
dataset_splitted_path = '../AMD_DataSet_pre'
train_data_path = os.path.join(dataset_splitted_path , 'train')
val_data_path = os.path.join(dataset_splitted_path , 'val')
test_data_path = os.path.join(dataset_splitted_path , 'test')
# Spiltting ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Adjusting Seed Value to 777
seed_constant = 777
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

# Variables for Augmentation Generators
dataset_augmented_path = '../AMD_DataSet_aug'
target_size = (448, 448)
batch_size = 64

# Variables for Compilation
OPTIMIZER = SGD(learning_rate=0.001)
loss = 'categorical_crossentropy'
model_name="AMD_SA_ResNet50_TF_FINAL"

# Variables for fitting
epochs = 10
batch_size = 64

In [ ]:
import splitfolders

# Function to split the data to train , val , test
def splitting_data_to_train_test_val(dataset_path , output_path , train_ratio , val_ratio , test_ratio , seed_value = 777):
    try:
        # This will include loading and splitting dataset
        splitfolders.ratio(dataset_path, output=output_path, seed=seed_value, ratio=(train_ratio, val_ratio, test_ratio))
        print(f"(✔) Splitting data is completed, with ratio of {train_ratio} for training ; {val_ratio} for validation ; {test_ratio} for testing.")
    except :
        print(f'(X) Error while splitting data.')
        return False
    return True

# Function to augment data (train , val) before training
def augmenting_data(train_data_path , val_data_path , output_path , target_size , batch_size , seed_value = 777):
    try:
        # Data Generators
        train_datagen = ImageDataGenerator(
            rescale=1./255.,
            horizontal_flip = True,
            vertical_flip = True,
            rotation_range = 50)
        
        val_datagen   = ImageDataGenerator(
            rescale=1./255)
        
        # Augmenting Data
        train_generator = train_datagen.flow_from_directory(
            train_data_path,
            seed=seed_value,
            target_size=target_size,
            batch_size=batch_size,
            class_mode = 'categorical')

        val_generator = val_datagen.flow_from_directory(
            val_data_path,
            seed=seed_value,
            target_size=target_size,
            batch_size=batch_size,
            class_mode = 'categorical')
        
        print(f"(✔) Augmenting data is Successfully done.")
    except :
        print(f'(X) Error while Augmenting data.')
        return None
    return train_generator , val_generator

# Function to evaluate the steps for the generator:
def evaluate_steps(generator):
    try:
        steps = generator.n // generator.batch_size
        print(f"(✔) Steps for the generator is evaluated {steps} steps. ")
    except :
        print(f'(X) Error while evaluating the generator steps.')
        return None
    return steps
    

In [ ]:
splitting_data_to_train_test_val(dataset_path , dataset_splitted_path , train_ratio , val_ratio , test_ratio)

In [ ]:
train_generator , val_generator = augmenting_data(train_data_path , val_data_path , dataset_augmented_path, target_size , batch_size , seed_constant)

## Snippet code integrated from 01_model_building

In [ ]:
# Global_Variables
inputs = Input(shape=(448, 448, 3))
sa_model_path = './models/MSE_MSLE_model001.h5'
pretrained_model = ResNet50(input_shape=(224, 224, 3),weights='imagenet', include_top=False)
final_model_name = 'AMD_SA_ResNet50_TF_FINAL'

In [ ]:
# Some defined Loss Functions used by the Scale Adaptive model (Encoder model)
def rmse(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def custom_metric_SSIM_(y_true, y_pred):
        x=y_true
        y=y_pred
        mean_x=tf.math.reduce_mean(x)
        mean_y=tf.math.reduce_mean(y)

        var_x=tf.math.reduce_variance(x)
        var_y=tf.math.reduce_variance(y)
        covar_xy=tfp.stats.covariance(x,y)

        k1=0.01
        k2=0.03
        l=float(tf.math.pow(2,20) - 1)
        c1=float(k1 * l)
        c2=float(k2 * l)
        print(type(c1))
        c1 = tf.math.square(c1)
        c2 = tf.math.square(c2)
      # SSIM=(2μxμy + c1)(2σxy +c2)/((μx)2+(μy)2 +c1)((σx)2 +(σy)2 + c2)
        m=tf.math.multiply(2.0,covar_xy)
        t1=tf.math.add(m , c2)
        m=tf.math.multiply(mean_x,mean_y)
        m=float(m * 2.0)
        t2=tf.math.add(m ,c1)
        t3=(tf.math.square(mean_x)+tf.math.square(mean_y) +c1)
        t4=(tf.math.square(var_x) +tf.math.square(var_y) + c2)

        SSIM=tf.math.divide(tf.math.multiply(t2,t1),tf.math.multiply(t3,t4))
        return tf.math.reduce_mean(SSIM)

def custom_loss_MSEhigh_MSLElow_(y, y_pred):
        l1=tf.keras.losses.MeanSquaredError()
        l_high=l1(y[:,:,:,:12],y_pred[:,:,:,:12])

        l2=tf.keras.losses.MeanSquaredLogarithmicError()
        l_low=l2(y[:,:,:,12:],y_pred[:,:,:,12:])

        return 0.4 * l_high + 0.6 * l_low

In [ ]:
# Function to load the encoder model or Scale Adaptive model
def load_encoder_model(model_path , inputs):
    try:
        resized_model = tf.keras.models.load_model(
            model_path,
            custom_objects={"custom_metric_SSIM_":custom_metric_SSIM_,
                            "custom_loss_MSEhigh_MSLElow_":custom_loss_MSEhigh_MSLElow_,
                            "rmse":rmse},
            compile=False)
        resized_model.trainable = False
        print(f"(✔) Model with name of '{model_path.split('/')[-1]}' is Successfuly Loaded....")
    except:
        print(f'(X) Cannot find the model in this path {model_path}')
        return None
    return resized_model(inputs) 

# Function to join the pretrained_model with the Scale Adaptive model for feature extraction
def build_feature_extractor_model(inputs , pretrained_model):
    # inputs here is the Scale Adaptive model
    try:
        feature_extractor_model = pretrained_model
        for layer in feature_extractor_model.layers:
            layer.trainable = True
        print(f"(✔) Successfully loaded the pretrained model for transfer learning and combine with the Scale Adaptive model.....")
    except:
        print(f'(X) Error while loading the pretrained model....')
        return None
    return feature_extractor_model(inputs)

# Function to attach the classification layers to classify our data
def attach_classifier(inputs):
    # inputs here is the feature extractor model which is (SA+ResNET50) so far
    try:
        X = GlobalAveragePooling2D()(inputs)
        X = Flatten()(X)

        X = Dense(units=512, activation='relu')(X)
        X = BatchNormalization()(X)
        X = Dropout(0.2)(X)

        X = Dense(units=256, activation='relu')(X)
        X = BatchNormalization()(X)
        X = Dropout(0.2)(X)

        X = Dense(units=128, activation='relu')(X)
        X = BatchNormalization()(X)
        X = Dropout(0.2)(X)

        X = Dense(units=4, activation='softmax', name=final_model_name)(X)
        print(f"(✔) Successfully attached the classification layers....")
    except:
        print(f'(X) Error while attaching the classification layers....')
        return None
    return X

In [ ]:
def building_model(sa_model_path , inputs ,pretrained_model):
    try:
        resized_model = load_encoder_model(sa_model_path,inputs)
        resized_pretrained_model = build_feature_extractor_model(inputs= resized_model[:,:,:,12:] , pretrained_model= pretrained_model)
        resized_pretrained_classification_model = attach_classifier(inputs= resized_pretrained_model)
        final_model = tf.keras.Model(inputs=inputs, outputs = resized_pretrained_classification_model)
        print(f"(✔) Final Model is Successfully Built....")
    except:
        print(f'(X) Error while building the model....')
        return None
    return final_model

In [ ]:
model = building_model(sa_model_path , inputs ,pretrained_model)
model.summary()

## Compiling the model

In [ ]:
# Function to compile the model
def compiling_model(model , optimizer , loss):
    try:
        model.compile(optimizer = optimizer,#'SGD',
                  loss = loss,
                  metrics = ['accuracy',tf.keras.metrics.AUC(),
                             tf.keras.metrics.Recall(), tf.keras.metrics.Precision(),
                             tf.keras.metrics.TruePositives(),tf.keras.metrics.TrueNegatives(),
                             tf.keras.metrics.FalsePositives(),tf.keras.metrics.FalseNegatives()])
        print(f"(✔) Adjusting compilaion step is completed....")
    except:
        print(f'(X) Error while compiling the model....')
        return None
    return model

# Function to fit the model "start training"
def fitting_model(model , epochs , batch_size , train_generator , val_generator):
    try:
        # Setting up some callback functions
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=50,verbose=1,factor=0.2)
        checkpoint = ModelCheckpoint('models/'+final_model_name+'.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
        early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)##
        
        # Evaluating Training and Evaluation Steps
        train_steps = evaluate_steps(train_generator)
        validation_steps = evaluate_steps(val_generator)
        
        # Start training or fitting model
        history = model.fit(
            train_generator,
            steps_per_epoch= train_steps,
            epochs=epochs,
            validation_data=val_generator,
            validation_steps=validation_steps,
            verbose = 1,
            callbacks=[reduce_lr, early_stop, checkpoint])

        print(f"(✔) Model is Trained Successfully....")
    except:
        print(f'(X) Error while training the model....')
        return None
    return model , history

In [ ]:
model = compiling_model(model , OPTIMIZER , loss)

In [ ]:
model , history = fitting_model(model , epochs , batch_size , train_generator , val_generator)

## Evaluating Model using "Testing Data"

In [ ]:
# Function to evaluate the model using the testing data
def evaluating_model(model , val_generator , test_generator):
    try:
        # Evaluating Model
        valid_loss, valid_accuracy, valid_auc, valid_recall, valid_precision, valid_true_positives, valid_true_negatives, valid_false_positives, valid_false_negatives = model.evaluate(val_generator)
        test_loss, test_accuracy, test_auc, test_recall, test_precision, test_true_positives, test_true_negatives, test_false_positives, test_false_negatives   = model.evaluate(test_generator)

        print('Validation Accuracy: ', round(valid_accuracy * 100, 1), "%")
        print('Test Accuracy: ', round(test_accuracy * 100, 1), "%")
        print(" ")
        print('Validation Specifity: ', round(valid_true_negatives / (valid_true_negatives + valid_false_positives) * 100, 1), "%")
        print('Test Specifity: ', round(test_true_negatives / (test_true_negatives + test_false_positives) * 100, 1), "%")
        print(" ")
        print('Validation sensitivity: ', round(valid_true_positives / (valid_true_positives+valid_false_negatives) * 100, 1), "%")
        print('Test sensitivity: ', round(test_true_positives / (test_true_positives + test_false_negatives) * 100, 1), "%")
        print(" ")
        print('Validation F1-score: ', round(2 * valid_true_positives / ((2 * valid_true_positives) + valid_false_positives + valid_false_negatives) * 100, 1), "%")
        print('Test F1-score: ', round(2 * test_true_positives / ((2 * test_true_positives) + test_false_positives + test_false_negatives) * 100, 1), "%")
        print(" ")
        print('Validation Loss: ', round(valid_loss, 1))
        print('Test Loss: ', round(test_loss, 1))
        print(f"(✔) Evaluation step is completed....")
    except:
        print(f'(X) Error while Evaluation the model....')
        return None
    return None

In [ ]:
# Data generator for test data
test_datagen   = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_path,
    seed=seed_constant,
    target_size=target_size,
    batch_size=batch_size,
    class_mode = 'categorical')

evaluating_model(model , val_generator , test_generator)